In [ ]:
import collections
import pathlib
import re
import string
 
import tensorflow as tf
 
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

ModuleNotFoundError: ignored

In [ ]:
dataset_dir = pathlib.Path("F:\Project\Dataset")
train_dir = dataset_dir/'Train'

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size)

Found 14000 files belonging to 2 classes.


In [ ]:
vali_dir=dataset_dir/'Validate'
raw_val_ds = preprocessing.text_dataset_from_directory(
    vali_dir,
    batch_size=batch_size)

Found 3000 files belonging to 2 classes.


In [ ]:
test_dir = dataset_dir/'Test'
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

Found 3001 files belonging to 2 classes.


In [ ]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [ ]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [ ]:
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [ ]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

In [ ]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [ ]:
text_batch, label_batch = next(iter(raw_train_ds))
first_tweet, first_label = text_batch[0], label_batch[0]
print("Tweet", first_tweet)
print("Label", first_label)

Tweet tf.Tensor(b' do you cuss alot?', shape=(), dtype=string)
Label tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

In [ ]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
438/438 [==============================] - 23s 51ms/step - loss: 1.0445 - accuracy: 0.6259 - val_loss: 0.6852 - val_accuracy: 0.7687
Epoch 2/10
438/438 [==============================] - 7s 15ms/step - loss: 0.6288 - accuracy: 0.8110 - val_loss: 0.5800 - val_accuracy: 0.7950
Epoch 3/10
438/438 [==============================] - 7s 15ms/step - loss: 0.5179 - accuracy: 0.8469 - val_loss: 0.5336 - val_accuracy: 0.8030
Epoch 4/10
438/438 [==============================] - 7s 15ms/step - loss: 0.4573 - accuracy: 0.8595 - val_loss: 0.5061 - val_accuracy: 0.8103
Epoch 5/10
438/438 [==============================] - 7s 15ms/step - loss: 0.4166 - accuracy: 0.8673 - val_loss: 0.4869 - val_accuracy: 0.8150
Epoch 6/10
438/438 [==============================] - 7s 15ms/step - loss: 0.3862 - accuracy: 0.8777 - val_loss: 0.4722 - val_accuracy: 0.8213
Epoch 7/10
438/438 [==============================] - 7s 15ms/step - loss: 0.3620 - accuracy: 0.8880 - val_loss: 0.4585 - val_accuracy: 0.823

In [ ]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [ ]:
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=2)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
438/438 [==============================] - 12s 26ms/step - loss: 0.5742 - accuracy: 0.6862 - val_loss: 0.4590 - val_accuracy: 0.8087
Epoch 2/5
438/438 [==============================] - 11s 26ms/step - loss: 0.3178 - accuracy: 0.8788 - val_loss: 0.3426 - val_accuracy: 0.8570
Epoch 3/5
438/438 [==============================] - 12s 26ms/step - loss: 0.1645 - accuracy: 0.9469 - val_loss: 0.3059 - val_accuracy: 0.8940
Epoch 4/5
438/438 [==============================] - 11s 26ms/step - loss: 0.0835 - accuracy: 0.9744 - val_loss: 0.3007 - val_accuracy: 0.9030
Epoch 5/5
438/438 [==============================] - 11s 26ms/step - loss: 0.0467 - accuracy: 0.9854 - val_loss: 0.3310 - val_accuracy: 0.9053


In [ ]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 660,738
Trainable params: 660,738
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

94/94 [==============================] - 1s 13ms/step - loss: 0.3689 - accuracy: 0.8940
Binary model accuracy: 82.57%
Int model accuracy: 89.40%


In [ ]:
export_model = tf.keras.Sequential(
    [int_vectorize_layer, int_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(int_accuracy))

94/94 [==============================] - 2s 13ms/step - loss: 0.3874 - accuracy: 0.8923
Accuracy: 89.40%


In [ ]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

In [ ]:
inputs = [
    "Fuck you its done",  # python
    "whatsup ma dude",  # java
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("tweet: ", input)
  print("Predicted label: ", label.numpy())

tweet:  Fuck you its done
Predicted label:  b'Trolls'
tweet:  whatsup ma dude
Predicted label:  b'Non-trolls'
